In [1]:
import pandas as pd
import numpy as np

import keras as K
import tensorflow as tf
import time

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
class Config(object):
    pass

config = Config()
config.log_dir = 'tf_logs/split_cnn/'+str(time.strftime('%Y-%m%d %H:%M:%S'))
config.batch_size = 50
config.epochs = 400
config.kernel_size = 3 # we will use 3x3 kernels throughout
config.pool_size = 2 # we will use 2x2 pooling throughout
config.drop_rate =0.25
config.conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
config.conv_depth_2 = 64 # ...switching to 64 after the first pooling layer

config.pool_strides = 2
config.conv_strides = 2

In [3]:
# fix random seed for reproducibility
np.random.seed(42)

In [4]:
from dance.audiosamples import RectifiedAudioSamples as AudioSamples

In [5]:
train_data = pd.read_pickle("train_data.pkl")
val_data = pd.read_pickle("val_data.pkl")
test_data = pd.read_pickle("test_data.pkl")

In [6]:
def generate_samples(train_data, val_data, test_data, sampler):
    try:
        del(train_samples)
        del(val_samples)
        del(test_samples)
    except:
        pass
    train_samples = sampler(train_data, subsample_rate=2)
    scaler = train_samples.norm()
    encoder = train_samples.encode()
    val_samples=sampler(val_data, subsample_rate=2)
    val_samples.norm(scaler)
    val_samples.encode(encoder)
    test_samples = sampler(test_data, subsample_rate=2)
    test_samples.norm(scaler)
    test_samples.encode(encoder)
    return train_samples, val_samples, test_samples

In [7]:
train, val, test = generate_samples(train_data, val_data, test_data, AudioSamples)

In [8]:
# tf.reset_default_graph()
#model = K.models.Sequential()
rate_input = K.layers.Input(shape=(1,))
wave_input = K.layers.Input(shape=((train.X.shape[1]-1,1,)))
                    
z = K.layers.Conv1D(config.conv_depth_1, config.kernel_size, name='w_con_1/conv')(wave_input)
z = K.layers.Activation('softplus', name='w_con_1/act')(z)
z = K.layers.MaxPooling1D(config.pool_size, config.pool_strides, name='w_con_1/pool')(z)
#z = K.layers.Dropout(config.drop_rate, name='w_con_1/drop')(z)

z = K.layers.Conv1D(config.conv_depth_2, config.kernel_size, activation='softplus', name='w_con_2/conv')(z)
# z = K.layers.Activation('softmax', name='w_con_2/act')(z)
z = K.layers.MaxPooling1D(config.pool_size, config.pool_strides, name='w_con_2/pool')(z)
#z = K.layers.BatchNormalization(name='w_con_2/batch_norm')(z)

z = K.layers.Flatten(name='flatten')(z)
x = K.layers.concatenate([rate_input, z], axis=1, name='concat')

x = K.layers.Dense(100, activation='softplus', kernel_regularizer=K.regularizers.l2(), name='dense')(x)
x = K.layers.Dense(train.Y.shape[-1], activation='softmax', kernel_regularizer=K.regularizers.l2(), name='loss')(x)


model = K.models.Model((rate_input, wave_input), x, name='split_cnn')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model._make_train_function()


In [9]:
tbCallback = K.callbacks.TensorBoard(config.log_dir, batch_size=config.batch_size, histogram_freq=10)
patience = K.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

history = model.fit([train.X[:,0], train.get_as_timeseries()[:,1:]], train.Y, epochs=config.epochs, 
                    batch_size=config.batch_size, 
                    validation_data=([val.X[:,0], val.get_as_timeseries()[:,1:]], val.Y), 
                    callbacks=[tbCallback, patience],
                    verbose=True,
                   )

Train on 39648 samples, validate on 10166 samples
INFO:tensorflow:Summary name w_con_1/conv/kernel:0 is illegal; using w_con_1/conv/kernel_0 instead.
INFO:tensorflow:Summary name w_con_1/conv/bias:0 is illegal; using w_con_1/conv/bias_0 instead.
INFO:tensorflow:Summary name w_con_2/conv/kernel:0 is illegal; using w_con_2/conv/kernel_0 instead.
INFO:tensorflow:Summary name w_con_2/conv/bias:0 is illegal; using w_con_2/conv/bias_0 instead.
INFO:tensorflow:Summary name dense/kernel:0 is illegal; using dense/kernel_0 instead.
INFO:tensorflow:Summary name dense/bias:0 is illegal; using dense/bias_0 instead.
INFO:tensorflow:Summary name loss/kernel:0 is illegal; using loss/kernel_0 instead.
INFO:tensorflow:Summary name loss/bias:0 is illegal; using loss/bias_0 instead.
Epoch 1/400
39648/39648 [==============================] - 606s - loss: 8.3293 - acc: 0.2352 - val_loss: 1.9101 - val_acc: 0.2392
Epoch 2/400
39648/39648 [==============================] - 441s - loss: 1.8115 - acc: 0.2992 - v

In [11]:
model.evaluate([test.X[:, 0], test.get_as_timeseries()[:, 1:]], y=test.Y, verbose=True)

18384/18384 [==============================] - 67s    


[2.1398003557850944, 0.1656875543951262]